In [31]:
import talib
import MetaTrader5 as mt  
import pandas as pd  
import time
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.optim import lr_scheduler
import random
from tqdm.auto import tqdm
from numpy import array
from pathlib import Path

In [32]:
mt.initialize()
login = 51225190
password = 'W5bu8LY1V'
server = 'Alpari-MT5-Demo'

mt.login(login, password, server)
# get account info
account_info = mt.account_info()
print(account_info)

# getting specific account data
login_number = account_info.login
balance = account_info.balance
equity = account_info.equity

AccountInfo(login=51225190, trade_mode=0, leverage=500, limit_orders=500, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=99.11, credit=0.0, profit=2.67, equity=101.78, margin=2.13, margin_free=99.65, margin_level=4778.403755868545, margin_so_call=50.0, margin_so_so=20.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Dneprovskii Vladimir', server='Alpari-MT5-Demo', currency='USD', company='Alpari')


In [124]:
device = "cpu"
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size=300, num_layers=1, device=device):
        super(LSTM, self).__init__()
        self.device = device
        # self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state        
        self.BatchNorm = nn.BatchNorm1d(32, affine=False)

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)
        self.fc_2 =  nn.Linear(hidden_size, 64) 
        self.fc_1 =  nn.Linear(64, 32) 
        self.fc = nn.Linear(32, 1) 
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device) #hidden state Variable(
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device) #internal state Variable(
        h_0 = self.BatchNorm(h_0)
        c_0 = self.BatchNorm(c_0)
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.dropout(out)
        out = self.fc_2(out) #first Dense
        out = self.relu(out) #relu
        out = self.dropout(out)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.dropout(out)
        out = self.fc(out) #Final Output
        return out

class CustomDataset(Dataset):
    def __init__(self, df_x):
        self.data_x = df_x         

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, index):
        feature = torch.from_numpy(self.data_x[index]).float()        
        return feature

input_size_M15 = 24

model_M15 = LSTM(input_size_M15, device=device)
model_M15.to(device)
PATH_M15 = "C:/Users/vovad/Desktop/Trading/models/model_M15_1/model_7.pth"
checkpoint_M15 = torch.load(PATH_M15, map_location=device)
lr_M15 = 0.00001
loss_fn = nn.BCEWithLogitsLoss().to(device)
optimizer_M15 = torch.optim.Adam(model_M15.parameters(), lr=lr_M15)
model_M15.load_state_dict(checkpoint_M15['model_state_dict'])
optimizer_M15.load_state_dict(checkpoint_M15['optimizer_state_dict'])
epoch_M15 = checkpoint_M15['epoch']
loss_M15 = checkpoint_M15['loss']

scaler_M15 = MinMaxScaler()
ct_M15 = ColumnTransformer(
        remainder='passthrough', 
        transformers=[
            ('mm', scaler_M15, [0,23])
        ])

input_size_M5 = 23

model_M5 = LSTM(input_size_M5, device=device)
model_M5.to(device)
PATH_M5 = "C:/Users/vovad/Desktop/Trading/models/model_M5_1/model_7.pth"
checkpoint_M5 = torch.load(PATH_M5, map_location=device)
lr_M5 = 0.00001
loss_fn = nn.BCEWithLogitsLoss().to(device)
optimizer_M5 = torch.optim.Adam(model_M5.parameters(), lr=lr_M5)
model_M5.load_state_dict(checkpoint_M5['model_state_dict'])
optimizer_M5.load_state_dict(checkpoint_M5['optimizer_state_dict'])
epoch_M5 = checkpoint_M5['epoch']
loss_M5 = checkpoint_M5['loss']

scaler_M5 = MinMaxScaler()
ct_M5 = ColumnTransformer(
        remainder='passthrough', 
        transformers=[
            ('mm', scaler_M5, [0,22])
        ])
        
def split_sequences(sequences, n_steps):
    X = []
    for i in range(len(sequences)):        
        end_ix = i + n_steps        
        if end_ix > len(sequences):
            break        
        seq_x = sequences[i:end_ix, :]
        X.append(seq_x)        
    return array(X)
    
def get_predictions(model, data_loader):
    model.eval()
    y_pred_list = []    
    with torch.inference_mode():
        for X_batch in data_loader:
            X_batch = X_batch.to(device)
            y_test_pred = model(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_pred_tag = torch.round(y_test_pred)
            y_pred_tag = y_pred_tag.long()
            y_pred_list.append(y_pred_tag.cpu().numpy())            

    predictions = [a.squeeze().tolist() for a in y_pred_list]
    predictions = sum(predictions, [])
    return predictions

def dataset_import_M15():    
    dataset = mt.copy_rates_from_pos("EURUSD_i", mt.TIMEFRAME_M15, 0, 722)    
    dataset = pd.DataFrame(dataset, columns=['open', 'high', 'low', 'close'])

    dataset['EMA_14'] = talib.EMA(dataset['close'], timeperiod=14)    
    dataset['EMA_30'] = talib.EMA(dataset['close'], timeperiod=30)
    dataset['EMA_48'] = talib.EMA(dataset['close'], timeperiod=48)   
    dataset['upperband'], dataset['middleband'], dataset['lowerband'] = talib.BBANDS(dataset['close'], timeperiod=24, nbdevup=8, nbdevdn=8, matype=0)    
    dataset['HT_TRENDLINE'] = talib.HT_TRENDLINE(dataset['close'])
    dataset['TEMA_24'] = talib.TEMA(dataset['close'], timeperiod=24)
    dataset['TEMA_30'] = talib.TEMA(dataset['close'], timeperiod=30)
    dataset['TEMA_48'] = talib.TEMA(dataset['close'], timeperiod=48)
    dataset['aroondown'], dataset['aroonup'] = talib.AROON(dataset['high'], dataset['low'], timeperiod=24)
    dataset['AROONOSC'] = talib.AROONOSC(dataset['high'], dataset['low'], timeperiod=24)
    dataset['HT_DCPERIOD'] = talib.HT_DCPERIOD(dataset['close'])
    dataset['HT_TRENDMODE'] = talib.HT_TRENDMODE(dataset['close'])    
    dataset['HT_SINE_sine'], dataset['HT_SINE_leadsine']  = talib.HT_SINE(dataset['close'])
    dataset['macd'], dataset['macdsignal'], dataset['macdhist'] = talib.MACD(dataset['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    dataset['RSI_24'] = talib.RSI(dataset['close'], timeperiod=24)
    dataset['NATR_30'] = talib.NATR(dataset['high'], dataset['low'], dataset['close'], timeperiod=30)
    dataset['WILLR_30'] = talib.WILLR(dataset['high'], dataset['low'], dataset['close'], timeperiod=30)

    dataset = dataset.dropna() 

    dataset = dataset.reset_index()
    dataset = dataset.drop(['index', 'open', 'high', 'low'], axis=1)
    dataset = dataset.astype('float32')
    dataset = ct_M15.fit_transform(dataset)
    n_steps = 550
    dataset = split_sequences(dataset, n_steps=n_steps)
    print(dataset.shape)
    test_dataset = CustomDataset(dataset)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last=False)
    pred = get_predictions(model_M15, test_dataloader)
    
    return pred, pred[-1] # pred[-3] if pred[-1]!=pred[-2] else pred[-1]

def dataset_import_M5():    
    dataset = mt.copy_rates_from_pos("EURUSD_i", mt.TIMEFRAME_M5, 0, 372)    
    dataset = pd.DataFrame(dataset, columns=['open', 'high', 'low', 'close'])

    dataset['EMA_14'] = talib.EMA(dataset['close'], timeperiod=14)    
    dataset['EMA_30'] = talib.EMA(dataset['close'], timeperiod=30)
    dataset['EMA_48'] = talib.EMA(dataset['close'], timeperiod=48)       
    dataset['upperband'], dataset['middleband'], dataset['lowerband'] = talib.BBANDS(dataset['close'], timeperiod=30, nbdevup=10, nbdevdn=10, matype=0)
    dataset['TEMA_14'] = talib.TEMA(dataset['close'], timeperiod=14)    
    dataset['TEMA_30'] = talib.TEMA(dataset['close'], timeperiod=30)
    dataset['TEMA_48'] = talib.TEMA(dataset['close'], timeperiod=48)
    dataset['aroondown'], dataset['aroonup'] = talib.AROON(dataset['high'], dataset['low'], timeperiod=24)    
    dataset['AROONOSC'] = talib.AROONOSC(dataset['high'], dataset['low'], timeperiod=24)
    dataset['HT_DCPERIOD'] = talib.HT_DCPERIOD(dataset['close'])
    dataset['HT_TRENDLINE'] = talib.HT_TRENDLINE(dataset['close'])
    dataset['HT_TRENDMODE'] = talib.HT_TRENDMODE(dataset['close'])    
    dataset['HT_SINE_sine'], dataset['HT_SINE_leadsine']  = talib.HT_SINE(dataset['close'])
    dataset['macd'], dataset['macdsignal'], dataset['macdhist'] = talib.MACD(dataset['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    dataset['RSI_24'] = talib.RSI(dataset['close'], timeperiod=24)    
    dataset['WILLR_30'] = talib.WILLR(dataset['high'], dataset['low'], dataset['close'], timeperiod=30)  
    
    dataset = dataset.dropna() 

    dataset = dataset.reset_index()
    dataset = dataset.drop(['index', 'open', 'high', 'low'], axis=1)
    dataset = dataset.astype('float32')
    dataset = ct_M5.fit_transform(dataset)
    n_steps = 200
    dataset = split_sequences(dataset, n_steps=n_steps)
    print(dataset.shape)  
    test_dataset = CustomDataset(dataset)
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last=False)
    pred = get_predictions(model_M5, test_dataloader)
    
    return pred, pred[-3] if pred[-1]!=pred[-2] else pred[-1]

In [213]:
X_M15, y_M15 = dataset_import_M15()
X_M5, y_M5 = dataset_import_M5()
y_M15, y_M5
print(X_M15, '\n', X_M5)

(32, 550, 24)
(32, 200, 23)
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
